# 🛍️ ShopSense — Phase 7: Evaluation

```

## Step 0 — Install Dependencies

In [1]:
import sys
!{sys.executable} -m pip install sentence-transformers faiss-cpu pandas numpy scikit-learn -q
print('✅ Dependencies installed.')


[notice] A new release of pip is available: 25.0 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
✅ Dependencies installed.


## Step 1 — Load Components

In [2]:
import json
import numpy as np
import pandas as pd
import faiss
from pathlib import Path
from sentence_transformers import SentenceTransformer

DATA_DIR = Path('data')

# Load embedding model
print('Loading model...')
embed_model = SentenceTransformer('all-MiniLM-L6-v2')

# Load FAISS product index
product_index = faiss.read_index(str(DATA_DIR / 'faiss_index.bin'))

# Load product metadata
with open(DATA_DIR / 'faiss_metadata.json') as f:
    product_metadata = json.load(f)

# Load reranked results from Phase 3 (Colab)
with open(DATA_DIR / 'sample_reranked_results.json') as f:
    reranked_results = json.load(f)

print(f'✅ Product index  : {product_index.ntotal:,} vectors')
print(f'✅ Products loaded: {len(product_metadata):,}')
print(f'✅ Reranked queries: {len(reranked_results)}')
print(f'\nQueries available for evaluation:')
for i, r in enumerate(reranked_results, 1):
    print(f'  {i}. "{r["query"]}"')

/Users/rithikabaskaran/MyFiles/ShopSense/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading model...


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 2198.88it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


✅ Product index  : 10,000 vectors
✅ Products loaded: 10,000
✅ Reranked queries: 8

Queries available for evaluation:
  1. "compact coffee maker for small kitchen"
  2. "non stick frying pan"
  3. "gift for someone who loves cooking"
  4. "storage solution for small apartment"
  5. "durable water bottle for gym"
  6. "budget friendly knife set for beginner cook"
  7. "air fryer for family of four"
  8. "cute kitchen decor for modern home"


## Step 2 — Build the Test Set

For evaluation we need **ground truth** — a set of queries where we
manually define which products are relevant.

We create relevance judgements on a 0-3 scale:
- **3** = highly relevant (exactly matches the query intent)
- **2** = relevant (matches the category and use case)
- **1** = partially relevant (related but not ideal)
- **0** = not relevant

We use the reranked results from Phase 3 as our candidate pool
and assign relevance scores based on how well each result matches.

In [3]:
def auto_judge_relevance(query: str, product: dict) -> int:
    """
    Automatically assigns a relevance score (0-3) to a product
    for a given query using keyword matching heuristics.

    This is a proxy for human judgement — good enough for a
    portfolio evaluation, not good enough for production.

    Scoring:
    3 — multiple query keywords in title AND product is right category
    2 — some query keywords in title or description
    1 — loosely related
    0 — not relevant
    """
    title = (product.get('title') or '').lower()
    desc  = (product.get('description') or '').lower()
    q     = query.lower()

    # Extract meaningful query words (3+ chars, skip stopwords)
    stopwords = {'for', 'the', 'and', 'with', 'under', 'best',
                 'good', 'nice', 'great', 'some', 'that'}
    q_words   = [w for w in q.split() if len(w) > 2 and w not in stopwords]

    # Count how many query words appear in title
    title_hits = sum(1 for w in q_words if w in title)
    desc_hits  = sum(1 for w in q_words if w in desc)

    total_words = len(q_words) if q_words else 1
    title_ratio = title_hits / total_words

    if title_ratio >= 0.6:
        return 3   # highly relevant
    elif title_ratio >= 0.3 or desc_hits >= 2:
        return 2   # relevant
    elif title_hits >= 1 or desc_hits >= 1:
        return 1   # partially relevant
    else:
        return 0   # not relevant


# Build test set from reranked results
test_queries = []

for result in reranked_results:
    query    = result['query']
    products = result['results']

    # Assign relevance scores to each result
    judgements = [
        {
            'asin'      : p['asin'],
            'title'     : p['title'],
            'relevance' : auto_judge_relevance(query, p),
            'rerank_score': p.get('rerank_score', 0),
            'faiss_score' : p.get('faiss_score', 0)
        }
        for p in products
    ]

    test_queries.append({
        'query'     : query,
        'judgements': judgements
    })

print(f'✅ Test set built: {len(test_queries)} queries')
print(f'\nSample judgements for first query:')
print(f'Query: "{test_queries[0]["query"]}"')
for j in test_queries[0]['judgements']:
    print(f'  [{j["relevance"]}] {j["title"][:60]}')

✅ Test set built: 8 queries

Sample judgements for first query:
Query: "compact coffee maker for small kitchen"
  [2] Small Coffee Table White Modern Wood Mini Rectangle Coffee T
  [2] Black & Decker DCM7 Cup-at-a-Time Personal 12-Ounce Coffeema
  [2] Famiworths Single Serve Coffee Maker for K Cup and Ground Co
  [2] Famiworths Single Serve Coffee Maker for K Cup and Ground Co
  [2] Nesco Blade Coffee Grinder


## Step 3 — Implement MRR and NDCG

Clean implementations of both metrics from scratch.

In [4]:
def compute_mrr(relevance_scores: list[int], threshold: int = 1) -> float:
    """
    Compute Reciprocal Rank for one query.

    Args:
        relevance_scores: List of relevance scores in ranked order
                         e.g. [0, 3, 2, 1, 0]
        threshold       : Minimum score to count as 'relevant'

    Returns:
        Reciprocal rank (1/position of first relevant result)
        0.0 if no relevant results found
    """
    for i, score in enumerate(relevance_scores, 1):  # positions are 1-indexed
        if score >= threshold:
            return 1.0 / i
    return 0.0   # no relevant result found


def compute_dcg(relevance_scores: list[int]) -> float:
    """
    Compute Discounted Cumulative Gain.
    Higher positions get more credit, lower positions get discounted.
    DCG = sum of (2^relevance - 1) / log2(position + 1)
    """
    return sum(
        (2**rel - 1) / np.log2(i + 2)   # i+2 because positions are 0-indexed here
        for i, rel in enumerate(relevance_scores)
    )


def compute_ndcg(relevance_scores: list[int]) -> float:
    """
    Compute Normalized DCG.
    Divides actual DCG by ideal DCG (perfect ranking).
    Result is always between 0 and 1.
    """
    actual_dcg = compute_dcg(relevance_scores)

    # Ideal DCG = DCG if results were sorted by relevance (best first)
    ideal_dcg  = compute_dcg(sorted(relevance_scores, reverse=True))

    if ideal_dcg == 0:
        return 0.0   # no relevant results at all

    return actual_dcg / ideal_dcg


def mean_metric(values: list[float]) -> float:
    """Average a list of metric values."""
    return sum(values) / len(values) if values else 0.0


# Quick sanity check
print('Sanity checks:')
print(f'  Perfect ranking  [3,3,2,1,0] NDCG = {compute_ndcg([3,3,2,1,0]):.4f} (expect 1.0)')
print(f'  Reversed ranking [0,1,2,3,3] NDCG = {compute_ndcg([0,1,2,3,3]):.4f} (expect low)')
print(f'  MRR first pos    [3,0,0,0,0] MRR  = {compute_mrr([3,0,0,0,0]):.4f} (expect 1.0)')
print(f'  MRR second pos   [0,3,0,0,0] MRR  = {compute_mrr([0,3,0,0,0]):.4f} (expect 0.5)')
print(f'  MRR no relevant  [0,0,0,0,0] MRR  = {compute_mrr([0,0,0,0,0]):.4f} (expect 0.0)')
print('\n✅ Metric implementations verified.')

Sanity checks:
  Perfect ranking  [3,3,2,1,0] NDCG = 1.0000 (expect 1.0)
  Reversed ranking [0,1,2,3,3] NDCG = 0.5884 (expect low)
  MRR first pos    [3,0,0,0,0] MRR  = 1.0000 (expect 1.0)
  MRR second pos   [0,3,0,0,0] MRR  = 0.5000 (expect 0.5)
  MRR no relevant  [0,0,0,0,0] MRR  = 0.0000 (expect 0.0)

✅ Metric implementations verified.


## Step 4 — Get FAISS-Only Results

We need to run the FAISS-only pipeline for all test queries
so we can compare it against the reranked results.

In [6]:
def faiss_search(query: str, top_k: int = 5) -> list[dict]:
    """Run FAISS-only search and return top_k results."""
    q_emb = embed_model.encode(
        [query],
        normalize_embeddings=True,
        convert_to_numpy=True
    ).astype('float32')

    scores, indices = product_index.search(q_emb, top_k)

    results = []
    for score, idx in zip(scores[0], indices[0]):
        if idx == -1:
            continue
        p = product_metadata[idx].copy()
        p['faiss_score'] = round(float(score), 4)
        results.append(p)
    return results


# Run FAISS search for all test queries
faiss_results = {}
for tq in test_queries:
    query              = tq['query']
    faiss_results[query] = faiss_search(query, top_k=5)

print(f'✅ FAISS search complete for {len(faiss_results)} queries')

✅ FAISS search complete for 8 queries


## Step 5 — Compute Metrics for Both Pipelines

In [7]:
rows = []

for tq in test_queries:
    query = tq['query']

    # ── FAISS-only pipeline ────────────────────────────────────
    faiss_products = faiss_results[query]
    faiss_rel      = [auto_judge_relevance(query, p) for p in faiss_products]
    faiss_mrr      = compute_mrr(faiss_rel)
    faiss_ndcg     = compute_ndcg(faiss_rel)

    # ── Reranked pipeline ──────────────────────────────────────
    # Results are already ranked by rerank_score from Phase 3
    reranked_products = next(
        (r['results'] for r in reranked_results if r['query'] == query), []
    )
    rerank_rel  = [auto_judge_relevance(query, p) for p in reranked_products]
    rerank_mrr  = compute_mrr(rerank_rel)
    rerank_ndcg = compute_ndcg(rerank_rel)

    # ── Compute improvement ────────────────────────────────────
    mrr_delta  = rerank_mrr  - faiss_mrr
    ndcg_delta = rerank_ndcg - faiss_ndcg

    rows.append({
        'query'           : query,
        'faiss_mrr'       : round(faiss_mrr,  4),
        'rerank_mrr'      : round(rerank_mrr, 4),
        'mrr_delta'       : round(mrr_delta,  4),
        'faiss_ndcg'      : round(faiss_ndcg,  4),
        'rerank_ndcg'     : round(rerank_ndcg, 4),
        'ndcg_delta'      : round(ndcg_delta,  4),
        'faiss_relevances' : faiss_rel,
        'rerank_relevances': rerank_rel
    })

df_eval = pd.DataFrame(rows)
print(f'✅ Metrics computed for {len(df_eval)} queries')
df_eval[['query', 'faiss_mrr', 'rerank_mrr', 'faiss_ndcg', 'rerank_ndcg']].head(10)

✅ Metrics computed for 8 queries


,query,faiss_mrr,rerank_mrr,faiss_ndcg,rerank_ndcg
0,compact coffee maker for small kitchen,1.0,1.0,1.0000,1.0000
1,non stick frying pan,1.0,1.0,1.0000,1.0000
2,gift for someone who loves cooking,1.0,1.0,0.7604,0.9808
3,storage solution for small apartment,1.0,1.0,0.7745,1.0000
4,durable water bottle for gym,1.0,1.0,0.9438,1.0000
5,budget friendly knife set for beginner cook,1.0,1.0,1.0000,1.0000
6,air fryer for family of four,1.0,1.0,1.0000,1.0000
7,cute kitchen decor for modern home,1.0,1.0,0.7992,1.0000


## Step 6 — Results Table

In [8]:
# Compute aggregate metrics
mean_faiss_mrr   = mean_metric(df_eval['faiss_mrr'].tolist())
mean_rerank_mrr  = mean_metric(df_eval['rerank_mrr'].tolist())
mean_faiss_ndcg  = mean_metric(df_eval['faiss_ndcg'].tolist())
mean_rerank_ndcg = mean_metric(df_eval['rerank_ndcg'].tolist())

mrr_improvement  = ((mean_rerank_mrr  - mean_faiss_mrr)  / mean_faiss_mrr  * 100
                    if mean_faiss_mrr  > 0 else 0)
ndcg_improvement = ((mean_rerank_ndcg - mean_faiss_ndcg) / mean_faiss_ndcg * 100
                    if mean_faiss_ndcg > 0 else 0)

print('=' * 70)
print('                  SHOPSENSE EVALUATION RESULTS')
print('=' * 70)
print(f'{"Query":<42} {"FAISS":>6} {"Rerank":>7} {"FAISS":>7} {"Rerank":>7}')
print(f'{"":<42} {"MRR":>6} {"MRR":>7} {"NDCG":>7} {"NDCG":>7}')
print('─' * 70)

for _, row in df_eval.iterrows():
    q = row['query'][:40]
    print(f"{q:<42} {row['faiss_mrr']:>6.3f} {row['rerank_mrr']:>7.3f} "
          f"{row['faiss_ndcg']:>7.3f} {row['rerank_ndcg']:>7.3f}")

print('─' * 70)
print(f"{'MEAN':<42} {mean_faiss_mrr:>6.3f} {mean_rerank_mrr:>7.3f} "
      f"{mean_faiss_ndcg:>7.3f} {mean_rerank_ndcg:>7.3f}")
print('=' * 70)
print()
print(f'MRR  improvement  from reranking: {mrr_improvement:+.1f}%')
print(f'NDCG improvement from reranking: {ndcg_improvement:+.1f}%')

                  SHOPSENSE EVALUATION RESULTS
Query                                       FAISS  Rerank   FAISS  Rerank
                                              MRR     MRR    NDCG    NDCG
──────────────────────────────────────────────────────────────────────
compact coffee maker for small kitchen      1.000   1.000   1.000   1.000
non stick frying pan                        1.000   1.000   1.000   1.000
gift for someone who loves cooking          1.000   1.000   0.760   0.981
storage solution for small apartment        1.000   1.000   0.774   1.000
durable water bottle for gym                1.000   1.000   0.944   1.000
budget friendly knife set for beginner c    1.000   1.000   1.000   1.000
air fryer for family of four                1.000   1.000   1.000   1.000
cute kitchen decor for modern home          1.000   1.000   0.799   1.000
──────────────────────────────────────────────────────────────────────
MEAN                                        1.000   1.000   0.910   0.9

## Step 7 — Per-Query Analysis

Shows relevance scores for each position so you can see exactly
how reranking changed the ordering for each query.

In [9]:
print('PER-QUERY RELEVANCE BREAKDOWN')
print('(Relevance: 3=highly relevant, 2=relevant, 1=partial, 0=not relevant)\n')

for _, row in df_eval.iterrows():
    q           = row['query']
    f_rels      = row['faiss_relevances']
    r_rels      = row['rerank_relevances']
    improved    = '✅' if row['ndcg_delta'] > 0 else ('➡️ ' if row['ndcg_delta'] == 0 else '❌')

    print(f"{improved} Query: \"{q}\"")
    print(f"   FAISS only  : {f_rels}  MRR={row['faiss_mrr']:.3f}  NDCG={row['faiss_ndcg']:.3f}")
    print(f"   + Reranking : {r_rels}  MRR={row['rerank_mrr']:.3f}  NDCG={row['rerank_ndcg']:.3f}")
    print()

PER-QUERY RELEVANCE BREAKDOWN
(Relevance: 3=highly relevant, 2=relevant, 1=partial, 0=not relevant)

➡️  Query: "compact coffee maker for small kitchen"
   FAISS only  : [2, 2, 2, 2, 2]  MRR=1.000  NDCG=1.000
   + Reranking : [2, 2, 2, 2, 2]  MRR=1.000  NDCG=1.000

➡️  Query: "non stick frying pan"
   FAISS only  : [3, 3, 3, 3, 3]  MRR=1.000  NDCG=1.000
   + Reranking : [3, 3, 3, 3, 3]  MRR=1.000  NDCG=1.000

✅ Query: "gift for someone who loves cooking"
   FAISS only  : [1, 1, 2, 1, 2]  MRR=1.000  NDCG=0.760
   + Reranking : [2, 2, 1, 2, 1]  MRR=1.000  NDCG=0.981

✅ Query: "storage solution for small apartment"
   FAISS only  : [1, 1, 2, 2, 1]  MRR=1.000  NDCG=0.774
   + Reranking : [2, 2, 2, 2, 2]  MRR=1.000  NDCG=1.000

✅ Query: "durable water bottle for gym"
   FAISS only  : [3, 2, 3, 2, 3]  MRR=1.000  NDCG=0.944
   + Reranking : [3, 3, 3, 3, 2]  MRR=1.000  NDCG=1.000

➡️  Query: "budget friendly knife set for beginner cook"
   FAISS only  : [2, 2, 2, 2, 2]  MRR=1.000  NDCG=1.000
 

## Step 8 — Visual Summary

In [12]:
import sys
!{sys.executable} -m pip install matplotlib -q
print('✅ matplotlib installed')


[notice] A new release of pip is available: 25.0 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
✅ matplotlib installed


In [13]:
import matplotlib
matplotlib.use('Agg')  # non-interactive backend for VS Code
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

fig, axes = plt.subplots(1, 2, figsize=(14, 6))
fig.suptitle('ShopSense — FAISS Only vs FAISS + Reranking', fontsize=14, fontweight='bold')

queries_short = [q[:30] + '...' if len(q) > 30 else q
                 for q in df_eval['query']]
x = np.arange(len(queries_short))
w = 0.35

# ── MRR plot ──
ax1 = axes[0]
ax1.bar(x - w/2, df_eval['faiss_mrr'],  w, label='FAISS Only',     color='#4C72B0', alpha=0.85)
ax1.bar(x + w/2, df_eval['rerank_mrr'], w, label='FAISS + Rerank', color='#DD8452', alpha=0.85)
ax1.axhline(mean_faiss_mrr,  color='#4C72B0', linestyle='--', alpha=0.5, linewidth=1)
ax1.axhline(mean_rerank_mrr, color='#DD8452', linestyle='--', alpha=0.5, linewidth=1)
ax1.set_title(f'MRR  (Mean: FAISS={mean_faiss_mrr:.3f}, Rerank={mean_rerank_mrr:.3f})')
ax1.set_xticks(x)
ax1.set_xticklabels(queries_short, rotation=45, ha='right', fontsize=7)
ax1.set_ylim(0, 1.1)
ax1.set_ylabel('MRR Score')
ax1.legend()
ax1.grid(axis='y', alpha=0.3)

# ── NDCG plot ──
ax2 = axes[1]
ax2.bar(x - w/2, df_eval['faiss_ndcg'],  w, label='FAISS Only',     color='#4C72B0', alpha=0.85)
ax2.bar(x + w/2, df_eval['rerank_ndcg'], w, label='FAISS + Rerank', color='#DD8452', alpha=0.85)
ax2.axhline(mean_faiss_ndcg,  color='#4C72B0', linestyle='--', alpha=0.5, linewidth=1)
ax2.axhline(mean_rerank_ndcg, color='#DD8452', linestyle='--', alpha=0.5, linewidth=1)
ax2.set_title(f'NDCG  (Mean: FAISS={mean_faiss_ndcg:.3f}, Rerank={mean_rerank_ndcg:.3f})')
ax2.set_xticks(x)
ax2.set_xticklabels(queries_short, rotation=45, ha='right', fontsize=7)
ax2.set_ylim(0, 1.1)
ax2.set_ylabel('NDCG Score')
ax2.legend()
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plot_path = DATA_DIR / 'evaluation_plot.png'
plt.savefig(plot_path, dpi=150, bbox_inches='tight')
plt.show()
print(f'✅ Plot saved → {plot_path}')

✅ Plot saved → data/evaluation_plot.png


/var/folders/ml/9ppxtlpd5p59628p20mp4_dh0000gn/T/ipykernel_34628/2464790918.py:45: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## Step 9 — Save Evaluation Results

In [14]:
# Save results table (without the list columns)
df_save = df_eval.drop(columns=['faiss_relevances', 'rerank_relevances'])
csv_path = DATA_DIR / 'evaluation_results.csv'
df_save.to_csv(csv_path, index=False)
print(f'✅ Saved: {csv_path}')

✅ Saved: data/evaluation_results.csv


## Step 10 — README-Ready Summary
Copy this directly into your GitHub README evaluation section.

In [15]:
readme_table = f"""## 📊 Evaluation Results

Evaluated on {len(df_eval)} queries using automated relevance judgements (0-3 scale).
Metrics: MRR (Mean Reciprocal Rank) and NDCG@5 (Normalized Discounted Cumulative Gain).

| Pipeline | MRR | NDCG@5 |
|---|---|---|
| FAISS Only | {mean_faiss_mrr:.3f} | {mean_faiss_ndcg:.3f} |
| FAISS + Cross-Encoder Reranking | {mean_rerank_mrr:.3f} | {mean_rerank_ndcg:.3f} |
| **Improvement** | **{mrr_improvement:+.1f}%** | **{ndcg_improvement:+.1f}%** |

### Per-Query Breakdown

| Query | FAISS MRR | Rerank MRR | FAISS NDCG | Rerank NDCG |
|---|---|---|---|---|
"""

for _, row in df_eval.iterrows():
    readme_table += (
        f"| {row['query'][:45]} "
        f"| {row['faiss_mrr']:.3f} "
        f"| {row['rerank_mrr']:.3f} "
        f"| {row['faiss_ndcg']:.3f} "
        f"| {row['rerank_ndcg']:.3f} |\n"
    )

print(readme_table)

# Save to file
readme_path = DATA_DIR / 'evaluation_readme.md'
with open(readme_path, 'w') as f:
    f.write(readme_table)
print(f'✅ README section saved → {readme_path}')

## 📊 Evaluation Results

Evaluated on 8 queries using automated relevance judgements (0-3 scale).
Metrics: MRR (Mean Reciprocal Rank) and NDCG@5 (Normalized Discounted Cumulative Gain).

| Pipeline | MRR | NDCG@5 |
|---|---|---|
| FAISS Only | 1.000 | 0.910 |
| FAISS + Cross-Encoder Reranking | 1.000 | 0.998 |
| **Improvement** | **+0.0%** | **+9.7%** |

### Per-Query Breakdown

| Query | FAISS MRR | Rerank MRR | FAISS NDCG | Rerank NDCG |
|---|---|---|---|---|
| compact coffee maker for small kitchen | 1.000 | 1.000 | 1.000 | 1.000 |
| non stick frying pan | 1.000 | 1.000 | 1.000 | 1.000 |
| gift for someone who loves cooking | 1.000 | 1.000 | 0.760 | 0.981 |
| storage solution for small apartment | 1.000 | 1.000 | 0.774 | 1.000 |
| durable water bottle for gym | 1.000 | 1.000 | 0.944 | 1.000 |
| budget friendly knife set for beginner cook | 1.000 | 1.000 | 1.000 | 1.000 |
| air fryer for family of four | 1.000 | 1.000 | 1.000 | 1.000 |
| cute kitchen decor for modern home | 1.000 | 1